In [156]:
%run model.ipynb
%run data.ipynb

In [157]:
import torch.optim as optim
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
import time
import matplotlib.pyplot as plt

In [158]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [149]:
X_train, X_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.1 ,random_state=1)
len(y_train), len(y_valid)

(723471, 80386)

In [150]:
train_data_tensor = torch.tensor(X_train.values.astype('float32')).float()
train_label_tensor = torch.tensor(y_train).float()
valid_data_tensor = torch.tensor(X_valid.values.astype('float32')).float()
valid_label_tensor = torch.tensor(y_valid).float()

In [151]:
train_tensor = torch.utils.data.TensorDataset(train_data_tensor, train_label_tensor)
valid_tensor = torch.utils.data.TensorDataset(valid_data_tensor, valid_label_tensor)
train_loader = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True, pin_memory= True , num_workers=8)
valid_loader = torch.utils.data.DataLoader(dataset = valid_tensor, batch_size = 40, shuffle = True, pin_memory= True, num_workers= 8)

In [152]:
model = model(train_data_tensor.shape[1])
model = Model.to(device)
summary(model, (batch_size, train_data_tensor.shape[1]))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1              [-1, 32, 100]           1,300
              ReLU-2              [-1, 32, 100]               0
            Linear-3                [-1, 32, 1]             101
Total params: 1,401
Trainable params: 1,401
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.01
Estimated Total Size (MB): 0.06
----------------------------------------------------------------


In [153]:
loss = RMSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr= 0.001)
Epoch = 10

In [155]:
train_loss_list = []
val_loss_list = []
for epoch in range(Epoch):
    since = time.time()
    total_train_loss = 0
    loss_val = 0
    for data, label in train_loader:
        data = data.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss_val = loss(out, label)
        total_train_loss += loss_val
        loss_val.backward()
        optimizer.step()
    for data, label in valid_loader:
        data = data.to(device)
        label = label.to(device)
        out = model(data)
        loss_val += loss(out,label)
    train_loss_list.append(total_train_loss.item() / len(train_loader))
    val_loss_list.append(loss_val.item() / len(valid_loader))
    now_time = time.time() - since
    print("epoch = ", epoch + 1, " train loss = ", train_loss_list[-1],
    " validation loss = ", val_loss_list[-1])
    print("Training time is:{:.0f}m {:.0f}s".format(now_time // 60, now_time % 60))

epoch =  1  train loss =  80.22785174045734  validation loss =  80.1208644278607
Training time is:1m 3s
epoch =  2  train loss =  80.2290625414658  validation loss =  80.14690609452737
Training time is:1m 0s
epoch =  3  train loss =  80.22758083064267  validation loss =  80.11383706467662
Training time is:1m 2s
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x00000200FFDDC820>
Traceback (most recent call last):
  File "C:\Users\Gevin\anaconda3\lib\site-packages\torch\utils\data\dataloader.py", line 1101, in __del__
    self._shutdown_workers()
  File "C:\Users\Gevin\anaconda3\lib\site-packages\torch\utils\data\dataloader.py", line 1075, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "C:\Users\Gevin\anaconda3\lib\multiprocessing\process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "C:\Users\Gevin\anaconda3\lib\multiprocessing\popen_spawn_win32.py", line 108, in wait
    res = _winapi.WaitForSingleObject(int

KeyboardInterrupt: 

In [145]:
plt.plot(range(1, Epoch + 1), train_loss_list)
plt.title("train")
plt.ylabel("loss value")
plt.xlabel("epoch")
plt.show()

NameError: name 'loss_val_list' is not defined

In [ ]:
plt.plot(range(1, Epoch + 1), val_loss_list)
plt.title("validation")
plt.ylabel("loss value")
plt.xlabel("epoch")
plt.show()